In [0]:
import csv
import math
import cv2
from PIL import Image
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon

In [0]:
ALPHA = 0.35

# 96, 128, 160, 192, 224
IMAGE_SIZE = 96
EPOCHS = 20
BATCH_SIZE = 32
PATIENCE = 50
MULTI_PROCESSING = True
THREADS = 4

In [0]:
TRAIN_CSV = "drive/flipkart/training_try_1.csv"  #img_dir='drive/inter_iit/conv_data/sat_match2'
VALIDATION_CSV = "drive/flipkart/validation_try_1.csv"
path2="drive/flipkart/train/"
model_path="drive/flipkart/model_try_1.hd5"

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 111313 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
class DataGenerator(Sequence):

    def __init__(self, csv_file):
        self.paths = []

        with open(csv_file, "r") as file:
            self.coords = np.zeros((sum(1 for line in file), 4))
            file.seek(0)

            reader = csv.reader(file, delimiter=",")
            real_index=0;
            for index, row in enumerate(reader):
                #for i, r in enumerate(row[1:5]):
                    #row[i+1] = int(r)
                if not index==0: 
                  
                  path=row[0];
                try:
                  im = Image.load(filename)
                  im.verify() #I perform also verify, don't know if he sees other types o defects
                  im.close() #reload is necessary in my case
                  im = Image.load(filename) 
                  im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                  im.close()
  
                  if index%100==0:
                    print (index)
                  x0=int(row[1]);
                  x1=int(row[2]);
                  y0=int(row[3])
                  y1=int(row[4]);
                  path=path2+path;
                 # path,x0, y0, x1, y1, _, _ = row
                 # print(index)
                  img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
                  image_height=int(img.shape[0])
                  image_width=int(img.shape[1])
                  self.coords[real_index, 0] = x0 * IMAGE_SIZE / image_width
                  self.coords[real_index, 1] = y0 * IMAGE_SIZE / image_height
                  self.coords[real_index, 2] = (x1 - x0) * IMAGE_SIZE / image_width
                  self.coords[real_index, 3] = (y1 - y0) * IMAGE_SIZE / image_height 
                  
                  self.paths.append(path)
                  real_index+=1;
                except:
                  continue

    def __len__(self):
        return math.ceil(len(self.coords) / BATCH_SIZE)

    def __getitem__(self, idx):
        batch_paths = self.paths[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE]
        batch_coords = self.coords[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE]

        batch_images = np.zeros((len(batch_paths), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        for i, f in enumerate(batch_paths):
            img = Image.open(f)
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            img = img.convert('RGB')

            batch_images[i] = preprocess_input(np.array(img, dtype=np.float32))
            img.close()

        return batch_images, batch_coords

In [0]:
class Validation(Callback):
    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        mse = 0
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_images, gt = self.generator[i]
            pred = self.model.predict_on_batch(batch_images)
            mse += np.linalg.norm(gt - pred, ord='fro') / pred.shape[0]

            pred = np.maximum(pred, 0)

            diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = np.maximum(diff_width, 0) * np.maximum(diff_height, 0)

            area_gt = gt[:,2] * gt[:,3]
            area_pred = pred[:,2] * pred[:,3]
            union = np.maximum(area_gt + area_pred - intersection, 0)

            intersections += np.sum(intersection * (union > 0))
            unions += np.sum(union)

        iou = np.round(intersections / (unions + epsilon()), 4)
        logs["val_iou"] = iou

        mse = np.round(mse, 4)
        logs["val_mse"] = mse

        print(" - val_iou: {} - val_mse: {}".format(iou, mse))


In [0]:
def create_model(trainable=False):
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = trainable

    x = model.layers[-1].output
    x = Conv2D(4, kernel_size=3, name="coords")(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)

In [0]:
    model = create_model()
  

In [0]:
 train_datagen = DataGenerator(TRAIN_CSV)

In [0]:
validation_datagen = Validation(generator=DataGenerator(VALIDATION_CSV))


In [0]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=[])
checkpoint = ModelCheckpoint("model-{val_iou:.2f}.h5", monitor="val_iou", verbose=1, save_best_only=True,
                               save_weights_only=True, mode="max", period=1)
stop = EarlyStopping(monitor="val_iou", patience=PATIENCE, mode="max")
reduce_lr = ReduceLROnPlateau(monitor="val_iou", factor=0.2, patience=10, min_lr=1e-7, verbose=1, mode="max")

#model.summary()

In [0]:
import numpy as np

In [28]:
len(train_datagen)

348

In [0]:
import tensorflow as tf
cp = tf.keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='jaccard_coef_int', save_best_only=True, verbose=1)

In [31]:
model.fit_generator(generator=train_datagen,
                      epochs=EPOCHS,validation_data=validation_datagen,
                      callbacks=[cp],
                      workers=THREADS,
                      use_multiprocessing=False,
                      shuffle=True,
                      verbose=1)
model.save(model_path)

Epoch 1/20


ValueError: ignored